In [1]:
import numpy as np
import matplotlib.pyplot as plt
import bilby
import pandas as pd
from gwpy.timeseries import TimeSeries
import scipy
from matplotlib.colors import LogNorm
from matplotlib.patches import Circle
import dynesty

/home/ljf/dis/bns/lib/python3.12/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS


In [2]:
duration = 4
sampling_frequency = 2048
t0_gps = 1126259460.4
n_samples = duration*sampling_frequency
time = np.linspace(0, duration, n_samples)

delta = 5
ras = np.radians(np.arange(0, 360, delta))
decs = np.radians(np.arange(-90, 90, delta))
ra_s = np.radians(np.linspace(-180, 180, len(ras)+1))
dec_s = np.radians(np.linspace(-90, 90, len(decs)+1))
radec_s = np.array([(ra, dec) for ra in ra_s for dec in dec_s])
radec = np.array([(ra, dec) for ra in ras for dec in decs])
radec_map = radec.reshape(36, 72, 2)

In [3]:
def cast_complex(df, col):
    df[f'{col}'] = np.array(df[f'{col}'].values, dtype=complex)

In [4]:
waveform = pd.read_csv('data/waveform_2.txt', sep=' ')
h1_strain = pd.read_csv('data/H1_strain_2.txt', sep=' ')
l1_strain = pd.read_csv('data/L1_strain_2.txt', sep=' ')
v1_strain = pd.read_csv('data/V1_strain_2.txt', sep=' ')
h1_psd = pd.read_csv('data/H1_psd_2.txt', sep=' ')
l1_psd = pd.read_csv('data/L1_psd_2.txt', sep=' ')
v1_psd = pd.read_csv('data/V1_psd_2.txt', sep=' ')

cast_complex(waveform, 'h+')
cast_complex(waveform, 'hx')
cast_complex(h1_strain, 'h')
cast_complex(l1_strain, 'h')
cast_complex(v1_strain, 'h')
cast_complex(h1_strain, 'white')
cast_complex(l1_strain, 'white')
cast_complex(v1_strain, 'white')

strain_dfs = [h1_strain, l1_strain, v1_strain]

ValueError: complex() arg is a malformed string

In [78]:
IFOH = bilby.gw.detector.InterferometerList(['H1'])[0]
IFOH.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(frequency_array=h1_psd['Hz'].values, psd_array=h1_psd['PSD'].values)
# IFOH.frequency_array = h1_psd['Hz']

IFOL = bilby.gw.detector.InterferometerList(['L1'])[0]
IFOL.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(frequency_array=l1_psd['Hz'].values, psd_array=l1_psd['PSD'].values)
# IFOL.frequency_array = l1_psd['Hz']

IFOV = bilby.gw.detector.InterferometerList(['V1'])[0]
IFOV.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(frequency_array=v1_psd['Hz'].values, psd_array=v1_psd['PSD'].values)
# IFOV.frequency_array = v1_psd['Hz']

IFOS = [IFOH, IFOL, IFOV]

In [77]:
prior = bilby.core.prior.PriorDict()
prior['geocent_time'] = bilby.core.prior.Uniform(name="geocent_time", minimum=t0_gps-0.1, maximum=t0_gps+0.1, unit='s')
prior['luminosity_distance'] = bilby.core.prior.Uniform(name="luminosity_distance", minimum=1, maximum=1000, unit='Mpc')
prior['ra'] = bilby.core.prior.Uniform(name="ra", minimum=0, maximum=2 * np.pi, unit='rad', boundary='periodic')
prior['dec'] = bilby.core.prior.Uniform(name="dec", minimum=-np.pi/2, maximum=np.pi/2, unit='rad', boundary='reflective')
prior['psi'] = bilby.core.prior.Uniform(name="psi", minimum=0, maximum=np.pi, unit='rad', boundary='periodic')

def log_likelihood(params, data_dfs, waveform, IFOS):
    DL = params['luminosity_distance']
    ra = params['ra']
    dec = params['dec']
    psi = params['psi']
    tc_geo = params['geocent_time']
    logl = 0

    for strain, IFO in zip(data_dfs, IFOS):
        fplus = IFO.antenna_response(ra, dec, tc_geo, psi, 'plus')
        fcross = IFO.antenna_response(ra, dec, tc_geo, psi, 'cross')
        template = waveform['h+'].values * fplus + waveform['hx'].values * fcross
        delay = IFO.time_delay_from_geocenter(ra, dec, tc_geo)
        h = template * np.exp(-1j * 2 * np.pi * waveform['Hz'].values * delay)
        h /= DL
        res = strain['h'] - h
        logl -= 0.5 * np.sum(np.conj(res)*res / IFO.power_spectral_density.psd_array.real)

    return logl

class Likelihood(bilby.core.likelihood.Likelihood):
    def __init__(self, data_dfs, waveform, IFOS):
        super().__init__()
        self.data_dfs = data_dfs
        self.waveform = waveform
        self.IFOS = IFOS

    def log_likelihood(self):
        return log_likelihood(self.data_dfs, self.waveform, self.IFOS)

gwl = Likelihood(data_dfs=[h1_strain, l1_strain, v1_strain], waveform=waveform, IFOS=[IFOH, IFOL, IFOV])

result = bilby.run_sampler(
    likelihood=gwl,
    priors=prior,
    sampler='dynesty',
    nlive=10,
    sample='rwalk',
    label='custom_likelihood'
)

16:24 bilby INFO    : Running for label 'custom_likelihood', output will be saved to 'outdir'


TypeError: 'NoneType' object is not iterable

In [ ]:
def priors(u):
    tc_geo = u[0] * (t0_gps + 0.1 - (t0_gps - 0.1)) + (t0_gps - 0.1)
    DL = u[1] * (1000 - 1) + 1
    ra = u[2] * (2 * np.pi - 0) + 0
    dec = u[3] * (np.pi/2 - (-np.pi/2)) + (-np.pi/2)
    psi = u[4] * (np.pi - 0) + 0
    return np.array([tc_geo, DL, ra, dec, psi])

def log_likelihood(params):
    tc_geo = params[0]
    DL = params[1]
    ra = params[2]
    dec = params[3]
    psi = params[4]
    logl = 0

    for strain, IFO in zip(data_dfs, IFOS):
        fplus = IFO.antenna_response(ra, dec, tc_geo, psi, 'plus')
        fcross = IFO.antenna_response(ra, dec, tc_geo, psi, 'cross')
        template = waveform['h+'].values * fplus + waveform['hx'].values * fcross
        delay = IFO.time_delay_from_geocenter(ra, dec, tc_geo)
        h = template * np.exp(-1j * 2 * np.pi * waveform['Hz'].values * delay)
        h /= DL
        res = strain['h'] - h
        logl -= 0.5 * np.sum(np.conj(res)*res / IFO.power_spectral_density.psd_array.real)

    return logl

In [ ]:
sampler_iso = dynesty.NestedSampler(isoll, prior_transform_iso, 21, nlive = 2000, sample = 'rwalk')
sampler_iso.run_nested(dlogz=0.01, print_progress=True)
res_iso = sampler_iso.results